<a href="https://colab.research.google.com/github/timmy-ops/DNNs_for_CLVs/blob/main/Replicated_tf1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/GoogleCloudPlatform/tensorflow-lifetime-value.git
! git clone https://github.com/timmy-ops/DNNs_for_CLVs

Cloning into 'tensorflow-lifetime-value'...
remote: Enumerating objects: 295, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 295 (delta 6), reused 0 (delta 0), pack-reused 281
Receiving objects: 100% (295/295), 301.79 KiB | 18.86 MiB/s, done.
Resolving deltas: 100% (145/145), done.
Cloning into 'DNNs_for_CLVs'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 10 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [2]:
#@title module imports
#title IMPORT MODULES
import pandas as pd
from datetime import datetime
import numpy as np

!pip install gast==0.2.2

#Statistical LTV
!pip install lifetimes
from lifetimes import BetaGeoFitter, GammaGammaFitter
from lifetimes.utils import calibration_and_holdout_data, summary_data_from_transaction_data

  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=1bb8c3a16887a3e8e479c9ca6523f34f5fb11aa0c2f833272041b6f4d63b5ab1
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.6.0 requires gast==0.4.0, but you have gast 0.2.2 which is incompatible.
tensorflow-probability 0.14.1 requires gast>=0.3.2, but you have gast 0.2.2 which is incompatible.
     |████████████████████████████████| 584 kB 30.7 MB/s 


In [3]:
#@title import data
#import xlsx
"""
df = pd.read_excel ('http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx')

print('The Data:')
print(df)
"""

"\ndf = pd.read_excel ('http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx')\n\nprint('The Data:')\nprint(df)\n"

In [4]:
#@title raw data types?
#Types?
"""
DataTypeSeries=df.dtypes
print('Datatype of the Columns:')
print(DataTypeSeries)
df.dtypes
"""

"\nDataTypeSeries=df.dtypes\nprint('Datatype of the Columns:')\nprint(DataTypeSeries)\ndf.dtypes\n"

In [5]:
#@title clean.sql { vertical-output: true }

threshold_date = pd.to_datetime('2011-08-08')
predict_end = pd.to_datetime('2011-12-12')
"""
#A-CIRCLE

#customer_id, order_date, order_value and order_qty_articles
df['customer_id'] = df['CustomerID']
df['order_qty_articles'] = df['Quantity']

df['order_date'] = pd.to_datetime(df['InvoiceDate']).dt.date

df['order_value'] = (df['UnitPrice']*df['Quantity']).round(2)                                                           

#tab1
tab1 = df.groupby(['customer_id','order_date']).agg(order_value = ('order_value','sum'), 
                                                   order_qty_articles = ('Quantity','sum')).reset_index()       #INDEX RESET
#tab1
tab2 = df.groupby('customer_id',).agg(latest_order=('order_date', 'max'))
tab2['latest_order'] = pd.to_datetime(tab2['latest_order'])

a = tab1.join(tab2, on='customer_id', how='inner')                             
                           
#B-CIRCLE
import numpy as np


# 39 - 52
tab1['positive_value'] = np.where(tab1['order_value'] > 0 , 1, 0)
                                    
past = tab1.loc[tab1['order_date'] < threshold_date]

# 34 - 54
tab3 = past.groupby('customer_id').agg({'positive_value':sum}).reset_index()                                    #INDEX RESET
tab3.rename(columns={'positive_value': 'cnt_positive_value'}, inplace=True)

# b (32-57)
mask3 = (tab3['cnt_positive_value'] > 1)
b = tab3.loc[mask3]

#JOIN A&B
join = pd.merge(a, b, on='customer_id', how='inner')

mask2 = (((join['order_qty_articles'] > 0) & (join['order_value'] > 0)) | ((join['order_qty_articles'] < 0) & (join['order_value'] < 0)) & (join['latest_order'] > pd.to_datetime('2011-09-12')))     #2011-09-12 = 3 Monate vor "heute"
data_cleaned = join.loc[mask2]

data_cleaned.to_csv('data_cleaned.csv')
"""

'\n#A-CIRCLE\n\n#customer_id, order_date, order_value and order_qty_articles\ndf[\'customer_id\'] = df[\'CustomerID\']\ndf[\'order_qty_articles\'] = df[\'Quantity\']\n\ndf[\'order_date\'] = pd.to_datetime(df[\'InvoiceDate\']).dt.date\n\ndf[\'order_value\'] = (df[\'UnitPrice\']*df[\'Quantity\']).round(2)                                                           \n\n#tab1\ntab1 = df.groupby([\'customer_id\',\'order_date\']).agg(order_value = (\'order_value\',\'sum\'), \n                                                   order_qty_articles = (\'Quantity\',\'sum\')).reset_index()       #INDEX RESET\n#tab1\ntab2 = df.groupby(\'customer_id\',).agg(latest_order=(\'order_date\', \'max\'))\ntab2[\'latest_order\'] = pd.to_datetime(tab2[\'latest_order\'])\n\na = tab1.join(tab2, on=\'customer_id\', how=\'inner\')                             \n                           \n#B-CIRCLE\nimport numpy as np\n\n\n# 39 - 52\ntab1[\'positive_value\'] = np.where(tab1[\'order_value\'] > 0 , 1, 0)\n           

In [6]:
#@title features_n-targets
"""
max_monetary = 15000

c = data_cleaned.groupby('customer_id').agg(order_date_min = ('order_date' , 'min'))
join = data_cleaned.join(c, on='customer_id', how='inner')


join['order_value_btyd'] = np.where(join['order_date'] == join['order_date_min'] , 
                                    0, 
                                    join['order_value'])

join['is_return'] = np.where(join['order_value'] < 1 , 
                                    1, 
                                    0)

  #filtern nach threshold
join_1 = join.loc[join['order_date'] < threshold_date]



  #aa['order_date_min'] = pd.to_datetime(c['order_date_min'])

tf = join_1.groupby('customer_id').agg(monetary_dnn = ('order_value', 'sum'),
                                     avg_basket_size = ('order_qty_articles','mean'),
                                     avg_basket_value = ('order_value','mean'),
                                     cnt_orders = ('order_date', 'nunique'),
                                     order_value_btyd_sum= ('order_value_btyd', 'sum'),
                                     order_date_min_bt= ('order_date_min', 'min'),
                                     cnt_returns= ('is_return', 'sum'),
                                     order_date_max_bt = ('order_date', 'max'))
                             
tf['monetary_btyd'] = np.where(tf['cnt_orders'] == 1,
                               0,
                               ( tf['order_value_btyd_sum'] / (tf['cnt_orders'] -1 )  ))                              

tf['recency'] = (pd.to_datetime(tf['order_date_max_bt']) - pd.to_datetime(tf['order_date_min_bt'])).dt.days
tf['T'] = (pd.to_datetime(threshold_date) - pd.to_datetime(tf['order_date_min_bt'])).dt.days

  #tt

  #in sql ausgeklammert: --- tt_0 = data_cleaned.loc[data_cleaned['order_date'] > threshold_date]
tt = data_cleaned.groupby('customer_id').agg(target_monetary = ('order_value', 'sum')).round(2)

tf_tt_0 = tf.join(tt, on='customer_id', how='inner').reset_index()


filter = ((tf_tt_0['monetary_dnn'] > 0) &
                    (tf_tt_0['monetary_btyd'] > 0) & 
                    (tf_tt_0['monetary_dnn'] <= max_monetary))   
tf_tt = tf_tt_0.loc[filter]

  #äusserstes select:

tf_tt['frequency_dnn'] = tf_tt['cnt_orders']
tf_tt['frequency_btyd'] = tf_tt['cnt_orders'] -1

  #ROUND
tf_tt['time_between'] = (tf_tt['recency'] / tf_tt['cnt_orders'])
tf_tt = tf_tt.round({"time_between": 2})


tf_tt = tf_tt.round({"avg_basket_value": 2, "avg_basket_size": 2})

tf_tt['has_returned'] = np.where(tf_tt['cnt_returns'] > 0,
                              1,
                              0)

tf_tt['frequency_btyd_clipped'] = np.where((tf_tt['cnt_orders'] -1) > 600,
                                        600,
                                        (tf_tt['cnt_orders'] -1) )

tf_tt['monetary_btyd_clipped'] = (np.where(tf_tt['monetary_btyd'] > 100000,
                                       100000,
                                       tf_tt['monetary_btyd'] )).round(2)


tf_tt['target_monetary_clipped'] = (np.where(tf_tt['target_monetary'] > 100000,     
                                         100000,
                                         tf_tt['target_monetary'] )).round(2)

#del (tf_tt['monetary_btyd'], tf_tt['frequency_btyd'], tf_tt['frequency_btyd_clipped'], tf_tt['monetary_btyd_clipped'], tf_tt['target_monetary_clipped'], 
#     tf_tt['target_monetary'], tf_tt['customer_id'], tf_tt['order_date_min_bt'], tf_tt['order_date_max_bt'], tf_tt['cnt_orders'], tf_tt['order_value_btyd_sum'])

tf_tt.to_csv('tf_tt.csv')
"""



'\nmax_monetary = 15000\n\nc = data_cleaned.groupby(\'customer_id\').agg(order_date_min = (\'order_date\' , \'min\'))\njoin = data_cleaned.join(c, on=\'customer_id\', how=\'inner\')\n\n\njoin[\'order_value_btyd\'] = np.where(join[\'order_date\'] == join[\'order_date_min\'] , \n                                    0, \n                                    join[\'order_value\'])\n\njoin[\'is_return\'] = np.where(join[\'order_value\'] < 1 , \n                                    1, \n                                    0)\n\n  #filtern nach threshold\njoin_1 = join.loc[join[\'order_date\'] < threshold_date]\n\n\n\n  #aa[\'order_date_min\'] = pd.to_datetime(c[\'order_date_min\'])\n\ntf = join_1.groupby(\'customer_id\').agg(monetary_dnn = (\'order_value\', \'sum\'),\n                                     avg_basket_size = (\'order_qty_articles\',\'mean\'),\n                                     avg_basket_value = (\'order_value\',\'mean\'),\n                                     cnt_orders = (\'o

In [7]:

#tf_tt = pd.read_csv('/content/DNNs_for_CLVs/features_n_targets.csv')


In [8]:
#tf_tt

In [9]:
#@title PARETO/NBD
#PARETO
"""
from lifetimes import BetaGeoFitter, ParetoNBDFitter, GammaGammaFitter
import math
from datetime import datetime
from lifetimes.plotting import plot_frequency_recency_matrix
from lifetimes.plotting import plot_probability_alive_matrix
from lifetimes.plotting import plot_period_transactions
import matplotlib.pyplot as plt

penalizer_coef = 0.01
discount_rate = 0.01

#column-selection
summary = tf_tt[['customer_id', 'frequency_btyd', 'recency', 'T',
                 'monetary_btyd']]
summary.columns = ['customer_id', 'frequency', 'recency', 'T',
                     'monetary_value']
summary = summary.set_index('customer_id')

actual_df = tf_tt[['customer_id', 'frequency_btyd', 'monetary_dnn',
                     'target_monetary']]
actual_df.columns = ['customer_id', 'train_frequency', 'train_monetary',
                       'act_target_monetary']

#PARETO/NBD fitter
paretof = ParetoNBDFitter(penalizer_coef= penalizer_coef)

paretof.fit(summary['frequency'], summary['recency'], summary['T'])



def predict_value(summary, actual_df, fitter, ggf, time_days, time_months):
  ltv = pd.DataFrame(data=np.zeros([actual_df.shape[0], 6]),
                     columns=['customer_id',
                              'actual_total',
                              'predicted_num_purchases',
                              'predicted_value',
                              'predicted_total',
                              'error'], dtype=np.float32)

  predicted_num_purchases = fitter.predict(time_days,
                                           summary['frequency'],
                                           summary['recency'],
                                           summary['T'])

  predicted_value = ggf.customer_lifetime_value(fitter,
                                                summary['frequency'],
                                                summary['recency'],
                                                summary['T'],
                                                summary['monetary_value'],
                                                time=time_months,
                                                discount_rate=discount_rate)

  ltv['customer_id'] = actual_df['customer_id']
  ltv['actual_total'] = actual_df['act_target_monetary']
  ltv['predicted_num_purchases'] = predicted_num_purchases.values
  ltv['predicted_value'] = predicted_value.values
  ltv['predicted_total'] = actual_df['train_monetary'] + ltv['predicted_value']
  ltv['error'] = ltv['actual_total'] - ltv['predicted_total']

  mse = pd.Series.sum(ltv['error'] * ltv['error']) / ltv.shape[0]
  rmse = math.sqrt(mse)


  plt.figure(figsize=(10, 7))
  plt.scatter(ltv['predicted_total'], ltv['actual_total'])
  plt.xlabel('Predicted')
  plt.ylabel('Actual')
  plt.title('Predicted vs Actual')
  plt.show()

  return ltv, rmse                        
                            
                            

train_end_date = pd.to_datetime(threshold_date, '%Y-%m-%d')
predict_end_date = pd.to_datetime(predict_end, '%Y-%m-%d')


fitter = paretof
  

time_days = (predict_end_date - train_end_date).days
time_months = int(math.ceil(time_days / 30.0))                                  #anzahl monate fuer die ggf.clv predicted

ggf = GammaGammaFitter(penalizer_coef=0)
ggf.fit(summary['frequency'], summary['monetary_value'])

ltv, rmse = predict_value(summary,actual_df,fitter,ggf,time_days,time_months)

#output_file = os.path.join(data_src, OUTPUT_FILE)
#ltv.to_csv(output_file, index=False)
print()                            
print('RMSE prediction error: %.2f' % rmse)    
print()
print()
print('RFM:')
print(summary)
#print(summary[['monetary_value', 'frequency']].corr())
#print(plot_frequency_recency_matrix(paretof))
#print(plot_probability_alive_matrix(paretof))
print(plot_period_transactions(paretof))    
print()                                                  
print(paretof.conditional_expected_number_of_purchases_up_to_time(1, summary['frequency'], summary['recency'], summary['T']))
"""

"\nfrom lifetimes import BetaGeoFitter, ParetoNBDFitter, GammaGammaFitter\nimport math\nfrom datetime import datetime\nfrom lifetimes.plotting import plot_frequency_recency_matrix\nfrom lifetimes.plotting import plot_probability_alive_matrix\nfrom lifetimes.plotting import plot_period_transactions\nimport matplotlib.pyplot as plt\n\npenalizer_coef = 0.01\ndiscount_rate = 0.01\n\n#column-selection\nsummary = tf_tt[['customer_id', 'frequency_btyd', 'recency', 'T',\n                 'monetary_btyd']]\nsummary.columns = ['customer_id', 'frequency', 'recency', 'T',\n                     'monetary_value']\nsummary = summary.set_index('customer_id')\n\nactual_df = tf_tt[['customer_id', 'frequency_btyd', 'monetary_dnn',\n                     'target_monetary']]\nactual_df.columns = ['customer_id', 'train_frequency', 'train_monetary',\n                       'act_target_monetary']\n\n#PARETO/NBD fitter\nparetof = ParetoNBDFitter(penalizer_coef= penalizer_coef)\n\nparetof.fit(summary['frequency'

In [10]:
#@title naive benchmark (neu bauen)
#BENCHMARKING
"""
tf_tt['feature_days'] = (pd.to_datetime(tf_tt['order_date_min_bt']) - threshold_date)
target_days = (threshold_date - predict_end)

tf_tt['predicted_monetary'] = tf_tt['avg_basket_value'] * (tf_tt['cnt_orders'] * (1 + (target_days/tf_tt['feature_days'])))

tf_tt['rsme_sub'] = tf_tt['predicted_monetary'] - tf_tt['target_monetary']

RMSE = ((((tf_tt['rsme_sub'].sum()) ** 2) / tf_tt.shape [0]) ** 0.5).round(2)

RMSE
"""


"\ntf_tt['feature_days'] = (pd.to_datetime(tf_tt['order_date_min_bt']) - threshold_date)\ntarget_days = (threshold_date - predict_end)\n\ntf_tt['predicted_monetary'] = tf_tt['avg_basket_value'] * (tf_tt['cnt_orders'] * (1 + (target_days/tf_tt['feature_days'])))\n\ntf_tt['rsme_sub'] = tf_tt['predicted_monetary'] - tf_tt['target_monetary']\n\nRMSE = ((((tf_tt['rsme_sub'].sum()) ** 2) / tf_tt.shape [0]) ** 0.5).round(2)\n\nRMSE\n"

In [11]:
#@title DNN: modules
#MODEL 
%tensorflow_version 1.x
import tensorflow as tf
from tensorflow import feature_column as tfc
from six import iteritems
import shutil

#only for the hypertune path fn:
import json
import os

TensorFlow 1.x selected.


In [12]:
#@title DNN: context.py

class CLVFeatures(object):

  HEADERS = ['customer_id', 'monetary_dnn', 'monetary_btyd', 'frequency_dnn',
             'frequency_btyd', 'recency', 'T', 'time_between',
             'avg_basket_value', 'avg_basket_size', 'cnt_returns',
             'has_returned', 'frequency_btyd_clipped', 'monetary_btyd_clipped',
             'target_monetary_clipped', 'target_monetary']
  HEADERS_DEFAULT = [[''], [0.0], [0.0], [0],
                     [0], [0], [0], [0.0],
                     [0.0], [0.0], [0],
                     [-1], [0], [0.0],
                     [0.0], [0.0]]
  NUMERICS = {
      'monetary_dnn': [],
      'recency': [],
      'frequency_dnn': [],
      'T': [],
      'time_between': [],
      'avg_basket_value': [],
      'avg_basket_size': [],
      'cnt_returns': []}
  CATEGORICALS_W_LIST = {
      'has_returned': [0, 1]}
  CROSSED = []
  KEY = 'customer_id'
  UNUSED = [KEY, 'monetary_btyd', 'frequency_btyd', 'frequency_btyd_clipped',
            'monetary_btyd_clipped', 'target_monetary_clipped']
  TARGET_NAME = 'target_monetary'

  def __init__(self, ignore_crosses=False): #, is_dnn=None

    #if not is_dnn:
    #  return

    self.ignore_crosses = ignore_crosses
    (self.headers, self.numerics_names, self.categorical_names) = self._keep_used()
    self.continuous, self.categorical = self._make_base_features()

    if not self.ignore_crosses:
      self.crossed_for_wide, self.crossed_for_deep = self._make_crossed()
  
  def _keep_used(self):
    headers = [h for h in self.HEADERS if h not in self.UNUSED]
    numerics_names = {
        k: v for k, v in iteritems(self.NUMERICS)
        if (k not in self.UNUSED) and (k != self.TARGET_NAME)
    }
    categorical_names = {
        k: v for k, v in iteritems(self.CATEGORICALS_W_LIST)
        if k not in self.UNUSED    
    }                          
    return headers, numerics_names, categorical_names
  
  def get_key(self):
    return self.KEY
  
  def get_used_headers(self, with_key=False, with_target=False):
    used_headers = [h for h in self.headers if h != self.TARGET_NAME]

    if with_key:
      used_headers.insert(0, self.KEY)
    if with_target:
      used_headers.append(self.TARGET_NAME)

    return used_headers

  def get_defaults(self, headers_names=None, with_key=False):
    if headers_names is None:
      headers_names = self.get_used_headers(with_key)

    keep_indexes = [self.HEADERS.index(n) for n in headers_names]
    return [self.HEADERS_DEFAULT[i] for i in keep_indexes]

  def get_all_names(self):
    return self.HEADERS

  def get_all_defaults(self):
    return self.HEADERS_DEFAULT

  def get_unused(self):
    return self.UNUSED

  def get_target_name(self):
    return self.TARGET_NAME

  def _make_base_features(self):
    continuous = {
        key_name: tfc.numeric_column(key_name)
        for key_name in self.numerics_names.keys()
    }
    categorical = {
        key_name: tfc.categorical_column_with_vocabulary_list(
            key=key_name,
            vocabulary_list=voc)
        for key_name, voc in self.categorical_names.items()
    }    
    return continuous, categorical

  def get_base_features(self):
    return self.continous, self.categorical

  def _prepare_for_crossing(self, key_name, num_bck, boundaries):
    key = None
    if key_name in self.continuous.keys():
      if boundaries is not None:
        key = tfc.bucketized_column(self.continuous[key_name], boundaries)
      else:
        key = tfc.categorical_column_with_identity(key_name, num_bck)
    elif key_name in self.categorical.keys():
      key = key_name
    else:
      key = key_name
    return key

  def _make_crossed(self):
    f_crossed_for_wide = []
    f_crossed_for_deep = []
    for to_cross in self.CROSSED:
      key = []
      bck_size = 1
      for (key, bck, bnd) in to_cross:
        keys.append(self._prepare_for_crossing(key, bck, bnd))
        bck_size *= bck

      t_crossed = tfc.crossed_column(keys, min(bck_size, 10000))
      t_dimension = int(bck_size**0.25)
      f_crossed_for_wide.append(t_crossed)
      f_crossed_for_deep.append(tfc.embedding_column(t_crossed, t_dimension))
    return f_crossed_for_wide, f_crossed_for_deep

  def get_wide_features(self):
    wide_features = self.categorical.values()
    if not self.ignore_crosses:
      wide_features += self.crossed_for_wide
    return wide_features  

  def get_deep_features(self, with_continuous=True):
    deep_features = [tfc.indicator_column(f) for f in self.categorical.values()]
    if with_continuous:
      deep_features += self.continuous.values()
    if not self.ignore_crosses:
      deep_features += self.crossed_for_deep
    return deep_features         

In [13]:
#@title Splits: import automatically from GitHub
#SPLIT Feature table by hashing

In [14]:
#@title DNN: model.py

clvf = CLVFeatures(ignore_crosses=True)

def parse_csv(csv_row):
  columns = tf.decode_csv(csv_row, record_defaults = clvf.get_all_defaults())
  features = dict(zip(clvf.get_all_names(), columns))
  
  for column_name in clvf.get_unused():
    features.pop(column_name)

  target = features.pop(clvf.get_target_name())

  return features, target


def dataset_input_fn(data_folder, prefix=None, mode=None, params=None, count=None):
  shuffle = True if mode == tf.estimator.ModeKeys.TRAIN else False

  filenames = tf.matching_files('{}{}*.csv'.format(data_folder, prefix))

  dataset = tf.data.TextLineDataset(filenames).skip(1)
  dataset = dataset.map(parse_csv)
  if shuffle:
    dataset = dataset.shuffle(buffer_size=params.buffer_size)
  dataset = dataset.repeat(count=count)
  dataset = dataset.batch(params.batch_size)

  iterator = tf.compat.v1.data.make_one_shot_iterator(dataset)#tf.compat.v1.data.make_one_shot_iterator(dataset)/#tf.compat.v1.data.make_initializable_iterator(dataset)
  
  features, target = iterator.get_next()

  return features, target

def read_train(data_folder, params):
  return dataset_input_fn(
      data_folder=data_folder,
      prefix='train',
      params=params,
      mode=tf.estimator.ModeKeys.TRAIN)


def read_eval(data_folder, params):
  return dataset_input_fn(data_folder=data_folder,
                          prefix='eval',
                          params=params)


def read_test(data_folder, params):
  return dataset_input_fn(data_folder=data_folder,
                          prefix='test',
                          params=params,
                          count=1)

def rmse_evaluator(labels, predictions):
  pred_values = predictions['predictions']
  return {'rmse': tf.metrics.root_mean_squared_error(labels, pred_values)} #tf.cast(labels, tf.float32)

def get_learning_rate(params):
  global_step = tf.train.get_global_step()
  learning_rate = tf.train.exponential_decay(                    
      learning_rate = params.learning_rate,
      global_step = global_step,                                                                
      decay_steps = params.checkpoint_steps,
      decay_rate = params.learning_decay_rate,
      staircase = True)
  return learning_rate

def get_optimizer(params):
  optimizer = tf.train.ProximalAdagradOptimizer(
      learning_rate = get_learning_rate(params),                                                
      l1_regularization_strength = params.l1_regularization,
      l2_regularization_strength = params.l2_regularization)
  return optimizer

def get_estimator(config, params, model_dir):    #estimator_name
  estimator = tf.estimator.DNNRegressor(
      feature_columns=clvf.get_deep_features(),
      hidden_units=params.hidden_units,
      config=config,
      model_dir=model_dir,
      optimizer=lambda: get_optimizer(params),
      batch_norm=True,
      dropout=params.dropout)
  
  estimator = tf.contrib.estimator.add_metrics(estimator, rmse_evaluator)       #delete contrib: where is the other contrib?
  return estimator

In [15]:
#@title DNN: task.py (Hyperparameter + parser.args)

TRAIN_SIZE = 883
NUM_EPOCHS = 70
BATCH_SIZE = 5
NUM_EVAL = 20

LEARNING_DECAY_RATE = 0.7
HIDDEN_UNITS = '128 64 32 16'
LEARNING_RATE = 0.00135
L1_REGULARIZATION = 0.0216647
L2_REGULARIZATION = 0.0673949
DROPOUT = 0.899732
SHUFFLE_BUFFER_SIZE = 10000

job_dir = '/content/model_checkpoint/'
data_src = '/content/DNNs_for_CLVs/'
ignore_crosses = False #default
learning_rate_decay = True

hypertune = False #adds numbers to ouputpath when turned on 'True'
resume = False #default (takes old savings for start if turned on 'True -> you may turn off hypertune then?)


def csv_serving_input_fn():
  clvf = CLVFeatures(ignore_crosses=True)
  used_headers = clvf.get_used_headers(with_key=True, with_target=False)
  default_values = clvf.get_defaults(used_headers)

  rows_string_tensor = tf.placeholder(dtype=tf.string, shape=[None],      #not compatible with egaer execution and tf.function?
                                      name='csv_rows')
  receiver_tensor = {'csv_rows': rows_string_tensor}

  row_columns = tf.expand_dims(rows_string_tensor, -1)
  columns = tf.decode_csv(row_columns, record_defaults=default_values)

  features = dict(zip(used_headers, columns))

  return tf.estimator.export.ServingInputReceiver(features, receiver_tensor)

In [16]:
#@title DNN: task.py (main execution)

tf.logging.set_verbosity(tf.compat.v1.logging.INFO)

if hypertune:
  config = json.loads(os.environ.get('TF_CONFIG', '{}'))
  trial = config.get('task', {}).get('trial', '')
  model_dir = os.path.join(job_dir, trial)
else:
  model_dir = job_dir

data_folder = '{}'.format(data_src)

train_steps = (TRAIN_SIZE/BATCH_SIZE) * NUM_EPOCHS
checkpoint_steps = int((TRAIN_SIZE/BATCH_SIZE) * (
      NUM_EPOCHS/NUM_EVAL))

config = tf.estimator.RunConfig(
    save_checkpoints_steps=checkpoint_steps
)

hidden_units = [int(n) for n in HIDDEN_UNITS.split()]

params = tf.contrib.training.HParams(
    num_epochs = NUM_EPOCHS,
    train_steps = train_steps,
    batch_size = BATCH_SIZE,
    hidden_units = hidden_units,
    learning_rate = LEARNING_RATE,
    ignore_crosses = ignore_crosses,
    buffer_size = SHUFFLE_BUFFER_SIZE,
    learning_rate_decay = learning_rate_decay,
    learning_decay_rate = LEARNING_DECAY_RATE,
    l1_regularization = L1_REGULARIZATION,
    l2_regularization = L2_REGULARIZATION,
    dropout= DROPOUT,
    checkpoint_steps = checkpoint_steps)

estimator = None

estimator = get_estimator(config=config,
                          params=params,
                          model_dir=model_dir)

train_spec = tf.estimator.TrainSpec(
    input_fn=lambda: read_train(data_folder, params),
    max_steps=train_steps)

eval_spec = tf.estimator.EvalSpec(
    input_fn=lambda: read_eval(data_folder, params),
    exporters=[
        tf.estimator.LatestExporter(
            name='estimate',
            serving_input_receiver_fn=csv_serving_input_fn,
            exports_to_keep=1,
            as_text=True
        )
    ],
    steps=1000,
    throttle_secs=1,
    start_delay_secs=1
)

if not resume:
    print('Removing previous trained model...')
    shutil.rmtree(model_dir, ignore_errors=True)
else:
    print('Resuming training...')

tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)
        
estimator.evaluate(lambda: read_test(data_folder, params), name="Test Set")

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Using config: {'_model_dir': '/content/model_checkpoint/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 618, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service'

{'average_loss': 17710520.0,
 'global_step': 12362,
 'label/mean': 3189.7307,
 'loss': 87154410.0,
 'prediction/mean': 0.2885886,
 'rmse': 4208.3867}

In [19]:
#@title DNN: Tensorboard
!pip install -U tensorboard-plugin-wit 
%load_ext tensorboard
%tensorboard --logdir=/content/model_checkpoint

Reusing TensorBoard on port 6006 (pid 115), started 0:04:43 ago. (Use '!kill 115' to kill it.)